In [1]:
with open('input.txt' , "r"  , encoding='utf-8') as f:
    text = f.read()
    



In [2]:
print(f"Length of text : {len(text)}")

Length of text : 1115394


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
unique_chars = sorted(list(set(text)))
print(f"Unique characters : {len(unique_chars)}")

Unique characters : 65


In [5]:
vocab_size = len(unique_chars)
print(f"Vocab size : {vocab_size}")

Vocab size : 65


In [6]:
print("".join(unique_chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [7]:
stoi = {ch: i for i, ch in enumerate(unique_chars)}
itos = {i: ch for i, ch in enumerate(unique_chars)}



In [8]:
print(stoi)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [9]:
encode = lambda s : [stoi[c] for c in s]
decode = lambda l : "".join([itos[i] for i in l])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [10]:
!nvidia-smi

Mon May  5 11:25:21 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:1B:00.0 Off |                  N/A |
|  0%   38C    P8              29W / 350W |     10MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
import torch
data = torch.tensor(encode(text) , dtype = torch.long)


In [13]:
data.shape

torch.Size([1115394])

In [14]:
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [15]:
n =  int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]



In [16]:
train_data.shape

torch.Size([1003854])

In [17]:
val_data.shape

torch.Size([111540])

In [18]:
block_size = 8

In [20]:
torch.manual_seed(1337)
batch_size = 32
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size, ))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y

xb , yb = get_batch('train')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"input: {context} target: {target}")
        

input: tensor([24]) target: 43
input: tensor([24, 43]) target: 58
input: tensor([24, 43, 58]) target: 5
input: tensor([24, 43, 58,  5]) target: 57
input: tensor([24, 43, 58,  5, 57]) target: 1
input: tensor([24, 43, 58,  5, 57,  1]) target: 46
input: tensor([24, 43, 58,  5, 57,  1, 46]) target: 43
input: tensor([24, 43, 58,  5, 57,  1, 46, 43]) target: 39
input: tensor([44]) target: 53
input: tensor([44, 53]) target: 56
input: tensor([44, 53, 56]) target: 1
input: tensor([44, 53, 56,  1]) target: 58
input: tensor([44, 53, 56,  1, 58]) target: 46
input: tensor([44, 53, 56,  1, 58, 46]) target: 39
input: tensor([44, 53, 56,  1, 58, 46, 39]) target: 58
input: tensor([44, 53, 56,  1, 58, 46, 39, 58]) target: 1
input: tensor([52]) target: 58
input: tensor([52, 58]) target: 1
input: tensor([52, 58,  1]) target: 58
input: tensor([52, 58,  1, 58]) target: 46
input: tensor([52, 58,  1, 58, 46]) target: 39
input: tensor([52, 58,  1, 58, 46, 39]) target: 58
input: tensor([52, 58,  1, 58, 46, 39, 

In [40]:
import torch.nn as nn
import torch.nn.functional as F
import torch 

torch.manual_seed(1338)

class BigramLanguageModel(nn.Module):
    
    def __init__(self , vocab_size):
        
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)
        
    def forward(self , idx , targets = None):
        
        logits = self.token_embedding_table(idx)
        
        
        if targets is None: 
            loss = None

        else :       
            B , T , C = logits.shape 
            logits = logits.view(B* T , C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits , targets)
        return logits , loss
    
    
    def generate(self , idx , max_new_tokens):
        
        for _ in range(max_new_tokens):
            logits , loss = self(idx)
            #print(f"LOGITS shape : {logits.shape}")
            logits = logits[:, -1 , :]
           # print(logits.shape)
            
            probs = F.softmax(logits , dim = -1)
            idx_next = torch.multinomial(probs , num_samples=1)
            
            idx = torch.cat((idx , idx_next) , dim = 1)
        return idx
        
    
model = BigramLanguageModel(vocab_size)
logits , loss  = model(xb , yb)


In [41]:
logits.shape

torch.Size([256, 65])

In [42]:
loss

tensor(4.5925, grad_fn=<NllLossBackward0>)

In [44]:
print(decode(model.generate(idx = torch.zeros((1 , 1) , dtype = torch.long) , max_new_tokens = 100)[0].tolist()))


n:$T!OS?uD;ydeTmf3h.RONMqKazzeTQjFrFx3aRMlieBpqKkSg: iSIFvnTyNUSVeyCFxvkSY.;NYC;N&'r$t,Db?jFS?MVrkO3


In [45]:
optimizer = torch.optim.AdamW(model.parameters() , lr = 1e-3)


In [47]:
for steps in range(10000):
    xb , yb = get_batch('train')
    logits , loss = model(xb , yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(f"Loss : {loss.item()}")
    

Loss : 2.5051445960998535


In [49]:
print(decode(model.generate(idx = torch.zeros((1 , 1) , dtype = torch.long) , max_new_tokens = 200)[0].tolist()))


ABRGehet,
d pevo th our weamy'
Ar, y y oul s at,-
Towe e bber cake ouk thathouchy beey&bewn ct t ch, Roacr
ind the y hf WivesanrobonXHe shaitw wn,
TII dere buk. an:
Wat buigrean?
THISoufoo p, s
Fr; ut
